In [1]:
import time
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

# The IBM Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Session, Sampler


# Plotting functions
import matplotlib.pyplot as plt
import json
from qiskit_optimization.applications.knapsack import Knapsack
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp,to_ising
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options
from qiskit_algorithms.optimizers import COBYLA,NFT
from qiskit_algorithms.minimum_eigensolvers import QAOA

In [2]:
service = QiskitRuntimeService()

The following method reads the data from the knapsack library formatted file, and uses the `qiskit` `Knapsack` classs to generate the Knapsack problem. It returs the Knapsack problem as a `QuadraticProgram`.

In [3]:
def read_data(filename):
    data_file = open(filename, 'r')
    count = 0
    p=[]
    w=[]
    while True:
        count += 1
        line=data_file.readline()
        if not line:
            break
        dp = [int(i) for i in line.rstrip('\n').split(' ')]
        if (count == 1):
            n=dp[0]
            b=dp[1]
        else:
            p.append(dp[0])
            w.append(dp[1])
    data_file.close()
    prob=Knapsack(p,w,b)
    qp=prob.to_quadratic_program()
    return qp 

## Read the data

Specify the data file and use the `read_data` method to get the problem as a `QuadraticProgram`. Print the quadratic program in the `lp` format.

In [4]:
data="kp_5_80"
qp=read_data('../../'+data)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Knapsack

Maximize
 obj: 33 x_0 + 24 x_1 + 36 x_2 + 37 x_3 + 12 x_4
Subject To
 c0: 15 x_0 + 20 x_1 + 17 x_2 + 8 x_3 + 31 x_4 <= 80

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End



## Get the Hamiltonian and the number of qubits required

Convert the `QudaraticProgram` to Ising Hamiltonian and get the number of required qubits.

In [5]:
qb=QuadraticProgramToQubo().convert(qp)
op,of = qb.to_ising()
min_qubits=op.num_qubits

In [6]:
min_qubits

12

## Use classical solver

Use `CPLEX` optimizer to find the optimal solution classically (as a baseline to compare to).

In [7]:
from qiskit_optimization.algorithms.cplex_optimizer import CplexOptimizer
solution=CplexOptimizer().solve(qp)

In [8]:
solution.fval

130.0

The optimal solution and the optimal objective are printed below.

In [9]:
print(solution.prettyprint())

objective function value: 130.0
variable values: x_0=1.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=0.0
status: SUCCESS


## Use quantum solver

Find the least buy device that has the required number of qubits and is not a simulator.

In [10]:
lbdevice=service.least_busy(simulator=False,min_num_qubits=min_qubits)

In [11]:
lbdevice.name

'ibm_cairo'

We can use the device obtained above or specify another device. Here we choose `ibmq_kolkata`. We select the optimization and resilience levels, as well as the execution shots as options, and use COBYLA as the optimizer for the QAOA algorithm with one layer (repetition parameter $p=1$). The optimal solution is obtained ib `best_bts` as a bitstring. 

In [12]:
device=lbdevice.name
#device='ibmq_kolkata'
backend = service.get_backend(device)
optimizer = COBYLA(maxiter=100)
options=Options()
options.optimization_level=3
options.resilience_level=1
options.execution.shots=2048
stime=time.time()
with Session(service=service, backend=backend):
    sampler = Sampler(options=options)
    qaoa = QAOA(sampler, optimizer, reps=1)
    result = qaoa.compute_minimum_eigenvalue(op)
total_time=time.time()-stime
best_bts = result.best_measurement['bitstring'][::-1]

We store the run data and the results in a dictionary.

In [13]:
#knapsack_results={}
td={}
td['data_set']=data
td['opt']=solution.fval
td['device']=device
td['optimization_level']=options.optimization_level
td['resilience_level']=options.resilience_level
td['shots']=options.execution.shots
td['total time']=total_time
td['bm_value']=np.real(-of-result.best_measurement['value'])
td['bm_bitstring']=result.best_measurement['bitstring']
td['bm_probability']=result.best_measurement['probability']
td['cost_function_evals']=result.cost_function_evals
td['optimizer_time']=result.optimizer_time

In [14]:
td

{'data_set': 'kp_5_80',
 'opt': 130.0,
 'device': 'ibm_cairo',
 'optimization_level': 3,
 'resilience_level': 1,
 'shots': 2048,
 'total time': 2468.2694618701935,
 'bm_value': 130.0,
 'bm_bitstring': '001010001111',
 'bm_probability': 0.0014932243741712574,
 'cost_function_evals': 26,
 'optimizer_time': 2441.86935710907}